# 说明


KNN是一种非常经典的非参数学习算法，相比于基于最小二乘的线性回归算法，KNN也可以看做一种基于X邻域的条件估计

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets as ds

%matplotlib inline

In [14]:
class Node(object):
    def __init__(self, axis, value, left, right, point):
        self._axis = axis
        self._value = value
        self._left = left
        self._right = right
        self._point = point
        
    @property
    def point(self):
        return self._point
    
    @property
    def value(self):
        return self._value
    
    @property
    def left(self):
        return self._left
    
    @property
    def right(self):
        return self._right
    
    @property
    def axis(self):
        return _axis
    
    
class NearestNeighbors(object):
    def __init__(self, n_neighbors=2, algorithm='kdtree'):
        self._all_algorithm = {
            'kdtree': self.kdtree
        }
        self._n_neighbors = n_neighbors
        self._algorithm = algorithm
        
        self._build_model = self._all_algorithm.get(algorithm, None)
        if self._build_model is None:
            raise Exception('Invalid algorithm')
        
    @property
    def n_neighbors(self):
        return self._n_neighbors
    
    @property
    def algorithm(self):
        return self._algorithm
    
    def kdtree(self):
        print('Begin to build kdtree model')
        
       
    
    def fix(self, X):
        self._X = X
        self._build_model()
    
    def kneighbors(self, X):
        pass
    
    
class KNNClassifier(NearestNeighbors):
    def __init__(self, *args, **kwargs):
        super(KNNClassifier, self).__init__(*args, **kwargs)
        
    
    
    

In [12]:
knn = KNNClassifier()

In [13]:
knn.n_neighbors

2